In [1]:
import numpy as np
from nltk.tokenize import RegexpTokenizer
from keras.models import Sequential, load_model
from keras.layers import LSTM
from keras.layers.core import Dense, Activation
from keras.optimizers import RMSprop
import heapq
import pickle


# usando o arquivo '1661-0.txt' que é muito utilizado para desenvolvimento de uma LSTM
path = '1661-0.txt'
text = open(path).read().lower()
print('quantidade de palavras: ', len(text))

quantidade de palavras:  581533


In [2]:
# Tokenização para apenas os caracteres alfanuméricos serem considerados como parte dos tokens 
# E cada token ser uma palavra do texto original guardada em word
tokenizer = RegexpTokenizer(r'\w+')
words = tokenizer.tokenize(text)

# Criação do dicionário de palavras únicas
# O NumPy é usado para realizar essa operação de forma eficiente

unique_words = np.unique(words)
unique_word_index = dict((c, i) for i, c in enumerate(unique_words))

# Essa variável representa o número de palavras consecutivas que serão usadas como entrada
WORD_LENGTH = 5

# Essas variáveis são usadas em um loop posterior para criar as sequências de palavras anteriores e suas respectivas palavras seguintes.
prev_words = []
next_words = []

# Geração dos pares de palavras de entrada e saída

for i in range(len(words) - WORD_LENGTH):
    prev_words.append(words[i:i + WORD_LENGTH])
    next_words.append(words[i + WORD_LENGTH])

print(prev_words[0], prev_words[1], prev_words[2])
print(next_words[0], next_words[1])


['the', 'project', 'gutenberg', 'ebook', 'of'] ['project', 'gutenberg', 'ebook', 'of', 'the'] ['gutenberg', 'ebook', 'of', 'the', 'adventures']
the adventures


In [3]:
# criamos matrizes vazias X e Y
# onde X será usada para armazenar a codificação one-hot das sequências de palavras anteriores 
# Y será usada para armazenar a codificação one-hot das palavras seguintes correspondentes.
X = np.zeros((len(prev_words), WORD_LENGTH, len(unique_words)), dtype=bool)
Y = np.zeros((len(next_words), len(unique_words)), dtype=bool)

# preencher as matrizes X e Y 
# com os valores da codificação one-hot das sequências de palavras anteriores e suas respectivas palavras seguintes
for i, each_words in enumerate(prev_words):
    for j, each_word in enumerate(each_words):
        X[i, j, unique_word_index[each_word]] = 1
    Y[i, unique_word_index[next_words[i]]] = 1


# Definição e compilação do modelo LSTM
model = Sequential()
model.add(LSTM(128, input_shape=(WORD_LENGTH, len(unique_words))))
model.add(Dense(len(unique_words)))
model.add(Activation('softmax'))

# definir o otimizador RMSprop como o otimizador para o modelo.
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Treinamento do modelo
history = model.fit(X, Y, validation_split=0.05, batch_size=128, epochs=5, shuffle=True)



/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/rmsprop.py:143: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/5
811/811 [==============================] - 20s 16ms/step - loss: 6.0140 - accuracy: 0.1074 - val_loss: 6.9978 - val_accuracy: 0.1057
Epoch 2/5
811/811 [==============================] - 12s 15ms/step - loss: 5.7603 - accuracy: 0.1455 - val_loss: 7.9987 - val_accuracy: 0.1084
Epoch 3/5
811/811 [==============================] - 11s 14ms/step - loss: 5.7662 - accuracy: 0.1747 - val_loss: 7.9584 - val_accuracy: 0.1020
Epoch 4/5
811/811 [==============================] - 12s 14ms/step - loss: 5.4171 - accuracy: 0.2088 - val_loss: 8.1269 - val_accuracy: 0.1068
Epoch 5/5
811/811 [==============================] - 12s 15ms/step - loss: 5.0635 - accuracy: 0.2499 - val_loss: 8.2898 - val_accuracy: 0.0962


In [18]:

# Função para preparar a entrada do modelo e prever as próximas palavras
def predict_completions(text, n=3):
    x = np.zeros((1, WORD_LENGTH, len(unique_words)))
    for t, word in enumerate(text.split()):
        x[0, t, unique_word_index[word]] = 1

    preds = model.predict(x, verbose=0)[0]
    
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)

    next_indices = heapq.nlargest(n, range(len(preds)), preds.take)
    return [unique_words[idx] for idx in next_indices]

# Teste de predição de palavras
setence = "Your life will never be the same again"
print("Correct sentence:", setence)
sequencia = " ".join(tokenizer.tokenize(setence.lower())[0:4])
print("Sequence:", sequencia)
print("Next possible words:", predict_completions(sequencia, 3))



Correct sentence: Your life will never be the same again
Sequence: your life will never
Next possible words: ['back', 'been', 'out']
